Weak Pin Bar: The volumes on a pin bar are important but the most important thing which determines if the pin bar is weak or not is the follow up bar and it's volume.

Hypothesis: If the follow up bar gives greater move than the pin bar but the volume was lower than the pin bar then, the low of the pin bar should get tested. This testing depends on the time frame of the pin bar. The results can be seen more quickly on intraday time frames so, we will be using 3, 5, and 15 minutes time frames on BTCUSDT and ETHUSDT futures contracts.